RV fits for IRAS04125+2902 system

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize
import emcee
import corner

In [ ]:
def rv_model( t, V, K, w = np.pi, e = 0, P = 8.834976, t0 = 1821.8257 + 2457000+ 8.834976*168 ):
    """
    Calculate the radial velocity curve of exoplanet system
    V      systemic velocity
    K      velocity semiamplitude
    w      longitude of periastron
    e      eccentricity
    P      orbital period
    t0     transit time
    t      array of times to output radial velocities
    """
    rv = 0*t + V
    
    f = np.pi/2 - w                                        
    ee = 2 * np.arctan(np.tan(f/2) * np.sqrt((1-e)/(1+e))) 
    tp = t0 - P/(2*np.pi) * (ee - e*np.sin(ee))        

    M = ( 2.0*np.pi / P ) * ( t - tp )
    M = M % (2.0*np.pi)

    E = 0*t
    tolerance = 1.0e-8
    max_iter = 100
    for i in range(len(t)):
        Ei = np.pi
        it = 0
        dE = 1
        while (it <= max_iter) and (np.abs(dE) > tolerance):
            dE = -(Ei - e*np.sin(Ei) - M[i])/(1.0 - e*np.cos(Ei))
            Ei += dE
            it += 1

        if np.abs(dE) > tolerance:
            print('Error: Newton-Raphson Iterative failed!')

        E[i] = Ei

    f = 2.0*np.arctan2( np.sqrt(1.0+e)*np.sin(E/2.0), np.sqrt(1.0-e)*np.cos(E/2.0) )
    
    rv += K * ( np.cos(f+w) + e*np.cos(w) )

    return rv

In [ ]:
# hpf_emp
t_emp = np.array([2460306.816606995, 2460307.820715443, 2460308.8080668016, 2460312.5735137896, 2460313.7949546175, 2460319.77257169])
vel_emp = np.asarray([10.68835501, -108.42860722,   21.56035341, -0.86798842,  -51.62865067, -123.9042143])
errvel_emp = np.asarray([26.96383118, 30.98179984, 34.91713508, 33.56414622, 35.50579489, 44.31492977])


Ms = 0.7
period = 8.834976
t_0 = 1821.8257 + 2457000
e = 0

In [ ]:
def ln_prior(theta):
    
    V, K, logf = theta
    
    if K < 0 or V < -60 or V > 0: 
        return -np.inf
    
    return 0

def ln_likelihood(theta, x, y, yerr):

    V, K, logf = theta
    model = rv_model( x, V, K)
    
    sigma2 = yerr**2 + model**2 * np.exp(2 * logf)
    
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(2 * np.pi *sigma2))

def ln_posterior(theta, x, y, yerr):
    ln_p = ln_prior(theta)
    if not np.isfinite(ln_p):
        return -np.inf
    return ln_p + ln_likelihood(theta, x, y, yerr)

def get_pos(nwalkers, ndim, guess):
    
    rng = np.random.default_rng()
    
    perturbation = [0.01, 0.01, 0.01] #V, K, w, e, logf
    
    pos = []
    while len(pos) < nwalkers:
        
        p = guess + perturbation*rng.standard_normal(ndim)
        
        if np.isfinite(ln_prior(p)):
            pos.append(p)
    
    return pos

In [ ]:
## initial guess values from my quick fits

thisT = t_emp
thisData = vel_emp
thisyerr = errvel_emp

V = np.median(thisData) 
K = 20 
mp = 0.1
w = np.pi
e = 0.0
P = period 
t0 = t_0 + period*168
t = np.arange(min(thisT)-period,max(thisT)+period,0.1)
data = rv_model(t, V,K,w,e)

nwalkers = 10
nsteps = 1000
burnin = 100


In [ ]:
# the mcmc part
nll = lambda *args: -ln_posterior(*args)
initial = [np.median(thisData),K,-1]

soln = minimize(nll, initial, args=(thisT, thisData, np.abs(thisyerr)))
ndim = np.size(initial)

V, K, logf = soln.x
model = rv_model( thisT, V, K)



theta_0 = soln.x
ndim, nwalkers = ndim, nwalkers
pos_0 = get_pos(nwalkers, ndim, theta_0)

sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, 
                                args=(thisT, thisData, thisyerr), threads=8)
sampler.run_mcmc(pos_0, nsteps, progress=False)

samples = sampler.get_chain()

V,K,logf = np.median(samples[:][-1], axis = 0)
Verr,Kerr,logferr = np.std(samples[:][-1], axis = 0)
tRange = np.arange(min(thisT)-period,max(thisT)+period,0.1)
model = rv_model( tRange, V, K)

samples = sampler.get_chain()
flat_samples = sampler.get_chain(discard=burnin, thin=15, flat=True)

In [ ]:
np.percentile((flat_samples[:,1]/ 203.255) * (P)**(1/3) * (np.random.normal(0.7, 0.04, len(flat_samples[:,1])))**(2/3), (50, 80, 95, 99))


In [ ]:
# float everything

In [ ]:
def ln_prior(theta):
    
    V, K, w, e, logf = theta
    
    if K < 0 or e < 0 or e > .9 or w < -np.pi or w > np.pi or V < -60 or V > 0:
        return -np.inf
    
    return 0

def ln_likelihood(theta, x, y, yerr):

    V, K, w, e, logf = theta
    model = rv_model( x, V, K, w, e)
    
    sigma2 = yerr**2 + model**2 * np.exp(2 * logf)
    
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(2 * np.pi *sigma2))

def ln_posterior(theta, x, y, yerr):
    ln_p = ln_prior(theta)
    if not np.isfinite(ln_p):
        return -np.inf
    return ln_p + ln_likelihood(theta, x, y, yerr)

def get_pos(nwalkers, ndim, guess):
    
    rng = np.random.default_rng()
    
    perturbation = [0.01, 0.01, 0.01, 0.01, 0.01] #V, K, w, e, logf
    
    pos = []
    while len(pos) < nwalkers:
        
        p = guess + perturbation*rng.standard_normal(ndim)
        
        if np.isfinite(ln_prior(p)):
            pos.append(p)
    
    return pos

In [ ]:
nll = lambda *args: -ln_posterior(*args)
initial = [-30,20,np.pi,0,-1]

soln = minimize(nll, initial, args=(thisT, thisData, np.abs(thisyerr)))
ndim = np.size(initial)

V, K, w, e, logf = soln.x
model = rv_model( thisT, V, K, w, e)



theta_0 = soln.x
ndim, nwalkers = ndim, nwalkers
pos_0 = get_pos(nwalkers, ndim, theta_0)

sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, 
                                args=(thisT, thisData, thisyerr), threads=8)
sampler.run_mcmc(pos_0, nsteps, progress=False)

samples = sampler.get_chain()

V, K, w, e,logf = np.median(samples[:][-1], axis = 0)
Verr,Kerr,werr,eerr,logferr = np.std(samples[:][-1], axis = 0)
tRange = np.arange(min(thisT)-period,max(thisT)+period,0.1)
model = rv_model( tRange, V, K, w,e)

samples = sampler.get_chain()
flat_samples = sampler.get_chain(discard=burnin, thin=15, flat=True)

In [ ]:
np.percentile((flat_samples[:,1]/ 203.255) * (P)**(1/3) * (np.random.normal(0.7, 0.04, len(flat_samples[:,1])))**(2/3), (50, 80, 95, 99))

In [ ]:
# with a linear trend
def ln_prior(theta):
    
    V, K, w, e, a, logf = theta
    
    if K < 0 or e < 0 or e > 1 or w < -np.pi or w > np.pi:
        return -np.inf
    
    return 0

def ln_likelihood(theta, x, y, yerr):

    V, K, w, e, a, logf = theta
    model = rv_model( x, V, K, w, e)
    timeoff = x#-np.min(x)
    model += a*timeoff
    
    sigma2 = yerr**2 + model**2 * np.exp(2 * logf)
    
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(2 * np.pi *sigma2))

def ln_posterior(theta, x, y, yerr):
    ln_p = ln_prior(theta)
    if not np.isfinite(ln_p):
        return -np.inf
    return ln_p + ln_likelihood(theta, x, y, yerr)

def get_pos(nwalkers, ndim, guess):
    
    rng = np.random.default_rng()
    
    perturbation = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01] #V, K, w, e, logf
    
    pos = []
    while len(pos) < nwalkers:
        
        p = guess + perturbation*rng.standard_normal(ndim)
        
        if np.isfinite(ln_prior(p)):
            pos.append(p)
    
    return pos

In [ ]:
nll = lambda *args: -ln_posterior(*args)
initial = [-30,20,np.pi,0, 0,-1]

soln = minimize(nll, initial, args=(thisT, thisData, np.abs(thisyerr)))
ndim = np.size(initial)

V, K, w, e,a, logf = soln.x
model = rv_model( thisT, V, K, w, e)+ a*thisT



theta_0 = soln.x
ndim, nwalkers = ndim, 20
pos_0 = get_pos(nwalkers, ndim, theta_0)

sampler = emcee.EnsembleSampler(nwalkers, ndim, ln_posterior, 
                                args=(thisT, thisData, thisyerr), threads=8)
sampler.run_mcmc(pos_0, nsteps, progress=False)

samples = sampler.get_chain()

V, K, w, e, a, logf = np.median(samples[:][-1], axis = 0)
Verr,Kerr,werr,eerr,aerr,logferr = np.std(samples[:][-1], axis = 0)
tRange = np.arange(min(thisT)-period,max(thisT)+period,0.1)
model = rv_model( tRange, V, K, w, e) + a*tRange

samples = sampler.get_chain()
flat_samples = sampler.get_chain(discard=burnin, thin=15, flat=True)

In [ ]:
np.percentile((flat_samples[:,1]/ 203.255) * (P)**(1/3) * (np.random.normal(0.7, 0.04, len(flat_samples[:,1])))**(2/3), (50, 80, 95, 99))